In [ ]:
import keras
import h5py
import numpy as np
import random
import cv2
from keras_tqdm import TQDMNotebookCallback
from tqdm import tqdm_notebook

infodir = 'C:/Science_Research/xraynet_data/images'
test_path = f'{infodir}/test.h5'
train_path = f'{infodir}/train.h5'
MODEL_NAME = 'xrayNet-keras-ResNet50' #gve the model a name
# input image dimensions

In [ ]:
from matplotlib import pyplot as plt
import h5py

h5f = h5py.File(test_path, 'r')
image = h5f['X'][11]
print(h5f['Y'][11])
plt.imshow(image, cmap='gray')
plt.show()
im = image*255
im = cv2.equalizeHist(im.astype('uint8'))
plt.imshow(im.astype('float32'), cmap='gray')
plt.show()
h5f.close()

In [ ]:
def load_val():
    h5f = h5py.File(test_path, 'r')
    X_val_pre = h5f['X']
    Y_val_pre = h5f['Y']

    X_val = []
    Y_val = []



    for image in tqdm_notebook(X_val_pre):
        im = image*255
        eq = cv2.equalizeHist(im.astype('uint8'))
        im = eq.astype('float32')/255
        X_val.append(cv2.cvtColor(im, cv2.COLOR_GRAY2RGB))

    for label in tqdm_notebook(Y_val_pre):
        Y_val.append(label[0])
        
    h5f.close()
    return X_val, Y_val
    
def load_data_batch(start,stop):
    h5f = h5py.File(train_path, 'r')
    X_train_pre = h5f['X'][start:stop]
    Y_train_pre = h5f['Y'][start:stop]

    X_train = []
    Y_train = []

    for image in tqdm_notebook(X_train_pre):
        im = image*255
        eq = cv2.equalizeHist(im.astype('uint8'))
        im = eq.astype('float32')/255
        X_train.append(cv2.cvtColor(im, cv2.COLOR_GRAY2RGB))

    for label in tqdm_notebook(Y_train_pre):
        Y_train.append(label[0])
        
    h5f.close()        
    return X_train, Y_train


In [ ]:
import os
if 'X_val1.npy' in os.listdir('./'):
    print('loading from file...')
    X_val = np.load('X_val.npy')
    Y_val = np.load('Y_val.npy')
    X_train = np.load('X_train.npy')
    Y_train = np.load('Y_train.npy')
    print('Data loaded')
else:
    X_val, Y_val = load_val()
    X_train, Y_train = load_data_batch(0,10000)
    np.save('X_val',X_val)
    np.save('Y_val',Y_val)
    np.save('X_train',X_train)
    np.save('Y_train',Y_train)


In [ ]:
X_train = np.array(X_train)
X_val = np.array(X_val)

Y_train = np.array(Y_train)
Y_val = np.array(Y_val)


print(X_train.shape, X_val.shape, Y_val.shape, Y_train.shape)

In [ ]:
Y_train = 1 - Y_train.astype('int')
Y_train = np.eye(2)[Y_train]

Y_val = 1 - Y_val.astype('int')
Y_val = np.eye(2)[Y_val]

In [ ]:
from keras import models
from keras import layers
from keras import optimizers
from keras import callbacks
from keras import backend as K

# Import the network architecture
import keras_resnet.models
IMG_WIDTH = 256
IMG_HEIGHT = 256
IMG_DEPTH = 3
BATCH_SIZE = 64
NB_EPOCHS = 15

shape, classes = (IMG_WIDTH, IMG_HEIGHT, IMG_DEPTH), 2


x = keras.layers.Input(shape)

model = keras_resnet.models.ResNet50(x, classes=classes)

model.compile(
    loss='binary_crossentropy',
    optimizer=optimizers.RMSprop(lr=2e-6),
    metrics=['acc'])

reduce_learning = callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,
    patience=2,
    verbose=1,
    mode='auto',
    epsilon=0.0001,
    cooldown=2,
    min_lr=0)

eary_stopping = callbacks.EarlyStopping(
    monitor='val_loss',
    min_delta=0,
    patience=7,
    verbose=1,
    mode='auto')

callbacks = [TQDMNotebookCallback()] #,eary_stopping, reduce_learning]

In [ ]:
# Train the the model
history = model.fit(
    X_train,
    Y_train,
    epochs=NB_EPOCHS,
    validation_data=(X_val, Y_val),
    verbose = 2,
    callbacks=callbacks
)

In [ ]:
import matplotlib.pyplot as plt

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)

plt.title('Training and validation accuracy')
plt.plot(epochs, acc, 'red', label='Training acc')
plt.plot(epochs, val_acc, 'blue', label='Validation acc')
plt.legend()

plt.figure()
plt.title('Training and validation loss')
plt.plot(epochs, loss, 'red', label='Training loss')
plt.plot(epochs, val_loss, 'blue', label='Validation loss')

plt.legend()

plt.show()

In [ ]:
max(val_acc)

### [0, 1] == Normal == 0

### [1, 0] == Abnormal == 1

### [Abnormalness, Normalness]